# 03. Economic potential estimation

Цель:
- загрузить результаты по годовой выработке (kWh/год)
- оценить CAPEX и OPEX PV-системы
- посчитать годовую экономию/доход
- вычислить срок окупаемости и NPV


In [1]:
import numpy as np
import geopandas as gpd
from pathlib import Path

In [2]:
PROJECT_ROOT = Path("/content/drive/MyDrive/solar_potential_project")
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_FINAL = PROJECT_ROOT / "data" / "final"
DATA_FINAL.mkdir(exist_ok=True)

In [3]:
gdf = gpd.read_file(DATA_PROCESSED / "estonia_tile_with_energy.geojson")
gdf.head()

,height,confidence,area_m2,usable_area_m2,installed_power_kw,annual_energy_kwh,geometry
0,-1.0,-1.0,13.637127,8.182276,1.472810,1189.293872,"POLYGON ((569001.273 6531328.82, 569001.812 65..."
1,-1.0,-1.0,10.178461,6.107077,1.099274,887.663590,"POLYGON ((560509.165 6534124.099, 560507.396 6..."
2,-1.0,-1.0,20.426216,12.255729,2.206031,1781.370255,"POLYGON ((573716.127 6529507.201, 573711.801 6..."
3,-1.0,-1.0,97.047615,58.228569,10.481142,8463.522500,"POLYGON ((572239.962 6528255.826, 572231.462 6..."
4,-1.0,-1.0,61.551992,36.931195,6.647615,5367.949216,"POLYGON ((575205.055 6530450.407, 575196.776 6..."


экономические параметры

In [5]:
# Цена установки (EUR за 1 кВт установленной мощности)
COST_PER_KW = 1200

# Ежегодные расходы на обслуживание как доля от CAPEX
OPEX_RATE = 0.01

# Цена электроэнергии (EUR за 1 kWh)
ELECTRICITY_PRICE = 0.20

# Деградация выработки PV в год (например 0.5%)
DEGRADATION = 0.005

# Горизонт расчёта (лет)
YEARS = 25

# Ставка дисконтирования (например 7%)
DISCOUNT_RATE = 0.07

CAPEX
$$ \text{CAPEX} = P_{\text{installed}} \cdot \text{cost}_{\text{per_kW}} $$

OPEX
$$ \text{OPEX} = \text{CAPEX} \cdot \text{opex_rate} $$

Годовая "экономия" (или доход)
$$ \text{Savings} = E_{\text{year}} \cdot \text{price} - \text{OPEX} $$

In [6]:
gdf["capex_eur"] = gdf["installed_power_kw"] * COST_PER_KW
gdf["opex_eur_per_year"] = gdf["capex_eur"] * OPEX_RATE

gdf["gross_value_eur_per_year"] = gdf["annual_energy_kwh"] * ELECTRICITY_PRICE
gdf["net_cashflow_year1_eur"] = gdf["gross_value_eur_per_year"] - gdf["opex_eur_per_year"]

простая окупаемость(payback)

$$ \text{Payback} = \frac{\text{CAPEX}}{\text{NetCashflow}_{\text{year1}}} $$

In [7]:
gdf["payback_years"] = gdf["capex_eur"] / gdf["net_cashflow_year1_eur"]

NPV (Net Present Value)
Расчёт денежных потоков с учётом деградации выработки

 Выработка энергии в год $t$:
$$ E_t = E_1 \cdot (1 - d)^{\,t-1} $$

 Денежный поток в год $t$:
$$ CF_t = E_t \cdot \text{price} - \text{OPEX} $$

Чистая приведённая стоимость (NPV):
$$ \text{NPV} = -\text{CAPEX} + \sum_{t=1}^{T} \frac{CF_t}{(1 + r)^t} $$

In [8]:
t = np.arange(1, YEARS + 1)

# коэффициенты деградации по годам: (1-d)^(t-1)
deg = (1 - DEGRADATION) ** (t - 1)

# коэффициенты дисконтирования: 1/(1+r)^t
disc = 1 / ((1 + DISCOUNT_RATE) ** t)

# annual_energy_kwh -> (n_buildings, 1) для broadcasting
E1 = gdf["annual_energy_kwh"].to_numpy()[:, None]

# CF_t для каждого здания и года
cashflows = (E1 * deg) * ELECTRICITY_PRICE - gdf["opex_eur_per_year"].to_numpy()[:, None]

npv = -gdf["capex_eur"].to_numpy() + (cashflows * disc).sum(axis=1)

gdf["npv_eur"] = npv

In [9]:
gdf[[
    "installed_power_kw",
    "annual_energy_kwh",
    "capex_eur",
    "net_cashflow_year1_eur",
    "payback_years",
    "npv_eur"
]].describe()

,installed_power_kw,annual_energy_kwh,capex_eur,net_cashflow_year1_eur,payback_years,npv_eur
count,2132.000000,2132.000000,2.132000e+03,2132.000000,2.132000e+03,2132.000000
mean,8.442778,6817.542887,1.013133e+04,1262.195246,8.026756e+00,3912.970286
std,24.876258,20087.578557,2.985151e+04,3719.000612,1.619115e-13,11529.388127
min,0.374092,302.079270,4.489104e+02,55.926750,8.026756e+00,173.380238
25%,2.042992,1649.715987,2.451590e+03,305.427294,8.026756e+00,946.864544
50%,4.611378,3723.687786,5.533654e+03,689.401021,8.026756e+00,2137.233297
75%,9.907761,8000.517183,1.188931e+04,1481.210302,8.026756e+00,4591.945592
max,940.839861,759728.187507,1.129008e+06,140655.559173,8.026756e+00,436050.623008


In [11]:
out = DATA_FINAL / "estonia_tile_with_economics.geojson"
gdf.to_file(out, driver="GeoJSON")

In [12]:
top = gdf.sort_values("npv_eur", ascending=False).head(10)[
    ["installed_power_kw", "annual_energy_kwh", "capex_eur", "payback_years", "npv_eur"]
]
top

,installed_power_kw,annual_energy_kwh,capex_eur,payback_years,npv_eur
1031,940.839861,759728.187507,1.129008e+06,8.026756,436050.623008
1164,422.027541,340787.239067,5.064330e+05,8.026756,195596.912622
1186,203.962840,164699.993244,2.447554e+05,8.026756,94530.564805
1059,166.218802,134221.682737,1.994626e+05,8.026756,77037.352755
915,147.093077,118777.659586,1.765117e+05,8.026756,68173.161551
1635,108.900901,87937.477168,1.306811e+05,8.026756,50472.250912
1075,106.636057,86108.616289,1.279633e+05,8.026756,49422.565066
1493,92.477136,74675.287677,1.109726e+05,8.026756,42860.336434
763,87.308041,70501.243151,1.047696e+05,8.026756,40464.618142
854,78.854420,63674.944518,9.462530e+04,8.026756,36546.622442
